In [3]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta

def get_data(symbol: str):
    # data = yf.download(tickers=symbol, period='300d', interval='1d')
    data = pd.read_csv("BTCUSD_Candlestick_1_D_ASK_08.05.2017-16.10.2021.csv")
    data.reset_index(inplace=True, drop=True)
    return data
# Get the data
data = get_data('BTC-USD')

In [4]:
# Calculate Bollinger Bands using pandas_ta
data.ta.bbands(length=10, std=1.5, append=True)

# Add the upper and lower bands to the DataFrame
data['Upper Band'] = data['BBU_10_1.5']
data['Lower Band'] = data['BBL_10_1.5']

def calculate_sma(data, length: int):
    return ta.sma(data['Close'], length)

# Calculate the moving average
data['SMA'] = calculate_sma(data, 20)
data.dropna(inplace=True)

In [5]:
data

,Gmt time,Open,High,Low,Close,Volume,BBL_10_1.5,BBM_10_1.5,BBU_10_1.5,BBB_10_1.5,BBP_10_1.5,Upper Band,Lower Band,SMA
19,01.06.2017 21:00:00.000,2413,2450,2374,2432,3.177806e+06,2165.504310,2319.8,2474.095690,13.302499,0.863588,2474.095690,2165.504310,2052.35
20,04.06.2017 21:00:00.000,2506,2630,2504,2625,1.808568e+06,2184.719837,2367.2,2549.680163,15.417385,1.206378,2549.680163,2184.719837,2100.70
21,05.06.2017 21:00:00.000,2690,2941,2690,2872,2.329196e+06,2157.191157,2432.7,2708.208843,22.650458,1.297252,2708.208843,2157.191157,2156.95
22,06.06.2017 21:00:00.000,2872,2886,2739,2739,3.008709e+06,2155.599042,2463.6,2771.600958,25.004137,0.947077,2771.600958,2155.599042,2206.30
23,07.06.2017 21:00:00.000,2740,2813,2612,2750,2.848845e+06,2157.844075,2491.8,2825.755925,26.804392,0.886578,2825.755925,2157.844075,2250.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1301,12.10.2021 21:00:00.000,55433,57543,54328,57047,2.013182e+03,50863.752663,54402.2,57940.647337,13.008471,0.873723,57940.647337,50863.752663,49377.80
1302,13.10.2021 21:00:00.000,57078,58568,56906,57505,2.001020e+03,52664.566400,55246.5,57828.433600,9.346958,0.937366,57828.433600,52664.566400,50101.50
1303,14.10.2021 21:00:00.000,57533,62922,56940,62587,2.235774e+03,52754.558444,56351.8,59949.041556,12.767087,1.366664,59949.041556,52754.558444,51094.85
1304,15.10.2021 21:00:00.000,62587,62608,60223,61077,1.943311e+03,52870.275433,56959.5,61048.724567,14.358358,1.003457,61048.724567,52870.275433,51964.40


In [6]:
def check_candles(data, backcandles, ma_column):
    categories = [0 for _ in range(backcandles)]
    for i in range(backcandles, len(data)):
        if all(data['Close'][i-backcandles:i] > data[ma_column][i-backcandles:i]):
            categories.append(2)  # Uptrend
        elif all(data['Close'][i-backcandles:i] < data[ma_column][i-backcandles:i]):
            categories.append(1)  # Downtrend
        else:
            categories.append(0)  # No trend
    return categories

# Apply the function to the DataFrame
data['Trend'] = check_candles(data, 7, 'SMA')

In [7]:
data

,Gmt time,Open,High,Low,Close,Volume,BBL_10_1.5,BBM_10_1.5,BBU_10_1.5,BBB_10_1.5,BBP_10_1.5,Upper Band,Lower Band,SMA,Trend
19,01.06.2017 21:00:00.000,2413,2450,2374,2432,3.177806e+06,2165.504310,2319.8,2474.095690,13.302499,0.863588,2474.095690,2165.504310,2052.35,0
20,04.06.2017 21:00:00.000,2506,2630,2504,2625,1.808568e+06,2184.719837,2367.2,2549.680163,15.417385,1.206378,2549.680163,2184.719837,2100.70,0
21,05.06.2017 21:00:00.000,2690,2941,2690,2872,2.329196e+06,2157.191157,2432.7,2708.208843,22.650458,1.297252,2708.208843,2157.191157,2156.95,0
22,06.06.2017 21:00:00.000,2872,2886,2739,2739,3.008709e+06,2155.599042,2463.6,2771.600958,25.004137,0.947077,2771.600958,2155.599042,2206.30,0
23,07.06.2017 21:00:00.000,2740,2813,2612,2750,2.848845e+06,2157.844075,2491.8,2825.755925,26.804392,0.886578,2825.755925,2157.844075,2250.95,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1301,12.10.2021 21:00:00.000,55433,57543,54328,57047,2.013182e+03,50863.752663,54402.2,57940.647337,13.008471,0.873723,57940.647337,50863.752663,49377.80,2
1302,13.10.2021 21:00:00.000,57078,58568,56906,57505,2.001020e+03,52664.566400,55246.5,57828.433600,9.346958,0.937366,57828.433600,52664.566400,50101.50,2
1303,14.10.2021 21:00:00.000,57533,62922,56940,62587,2.235774e+03,52754.558444,56351.8,59949.041556,12.767087,1.366664,59949.041556,52754.558444,51094.85,2
1304,15.10.2021 21:00:00.000,62587,62608,60223,61077,1.943311e+03,52870.275433,56959.5,61048.724567,14.358358,1.003457,61048.724567,52870.275433,51964.40,2


## 1 - Entry Based on Bollinger Bands - Candles Crossing

In [8]:
# Check conditions and assign entry values
data['entry'] = 0

# Condition for entry category 2 (buy entry)
buy_entry_condition = (data['Trend'] == 2) & ((data['Open'] < data['Lower Band']) & (data['Close'] > data['Lower Band']))
data.loc[buy_entry_condition, 'entry'] = 2

# Condition for entry category 1 (sell entry)
sell_entry_condition = (data['Trend'] == 1) & ((data['Open'] > data['Upper Band']) & (data['Close'] < data['Upper Band']))
data.loc[sell_entry_condition, 'entry'] = 1

In [9]:
data[data['entry']!=0]

,Gmt time,Open,High,Low,Close,Volume,BBL_10_1.5,BBM_10_1.5,BBU_10_1.5,BBB_10_1.5,BBP_10_1.5,Upper Band,Lower Band,SMA,Trend,entry
135,12.11.2017 22:00:00.000,5894,6828,5605,6508,3.286785e+06,6412.203907,6893.1,7373.996093,13.952970,0.099602,7373.996093,6412.203907,6329.15,2,2
189,28.01.2018 22:00:00.000,11828,11835,11095,11306,6.273800e+03,10630.492186,11141.9,11653.307814,9.179903,0.660439,11653.307814,10630.492186,12932.00,1,1
475,04.03.2019 22:00:00.000,3760,3930,3742,3894,2.412455e+05,3779.754569,3885.9,3992.045431,5.463107,0.538155,3992.045431,3779.754569,3769.55,2,2
569,14.07.2019 21:00:00.000,10700,11007,9920,10881,1.219458e+06,10740.494448,11569.0,12397.505552,14.322855,0.084795,12397.505552,10740.494448,11119.20,2,2
634,10.10.2019 21:00:00.000,8638,8867,8345,8398,7.621356e+05,8118.794825,8358.4,8598.005175,5.733279,0.582636,8598.005175,8118.794825,8910.60,1,1
721,23.01.2020 22:00:00.000,8439,8565,8278,8532,6.587761e+05,8519.970250,8729.1,8938.229750,4.791554,0.028761,8938.229750,8519.970250,8259.60,2,2
771,24.03.2020 21:00:00.000,6813,7034,6530,6657,5.944774e+05,5077.771397,5945.0,6812.228603,29.175058,0.910503,6812.228603,5077.771397,7275.95,1,1
1246,18.08.2021 21:00:00.000,44557,46935,43999,46655,1.137963e+03,44570.866615,46017.2,47463.533385,6.286056,0.720489,47463.533385,44570.866615,43853.00,2,2


In [22]:
import plotly.graph_objects as go

dfpl = data[:]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

# Add the moving averages to the plot
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['SMA'], mode='lines', name='SMA', line=dict(color='red')))
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['Lower Band'], mode='lines', name='Lower Band', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['Upper Band'], mode='lines', name='Upper Band', line=dict(color='pink')))

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

## 2 - Entry based on RSI and Bollinger Bands

In [11]:
def add_rsi_column(data):
    # Calculate RSI with a period of 14
    data['RSI'] = ta.rsi(data['Close'])
    return data

data = add_rsi_column(data)

In [12]:
def rsi_signal(data):
    data['RSI Signal'] = 0  # Initialize the signal column with 0

    # Set the signal category to 2 when the price is below the lower Bollinger Band and RSI is below 30
    data.loc[(data['Close'] < data['Lower Band']) & (data['RSI'] < 55), 'RSI Signal'] = 2

    # Set the signal category to 1 when the price is above the upper Bollinger Band and RSI is above 70
    data.loc[(data['Close'] > data['Upper Band']) & (data['RSI'] > 45), 'RSI Signal'] = 1

    return data

In [13]:
data = rsi_signal(data)

In [14]:
data[data["RSI Signal"]!=0]

,Gmt time,Open,High,Low,Close,Volume,BBL_10_1.5,BBM_10_1.5,BBU_10_1.5,BBB_10_1.5,BBP_10_1.5,Upper Band,Lower Band,SMA,Trend,entry,RSI,RSI Signal
35,25.06.2017 21:00:00.000,2473,2561,2256,2356,2.290450e+06,2381.179679,2583.2,2785.220321,15.641090,-0.062320,2785.220321,2381.179679,2579.10,0,0,42.073327,2
45,09.07.2017 21:00:00.000,2529,2541,2361,2385,1.802443e+06,2434.204944,2536.1,2637.995056,8.035571,-0.241449,2637.995056,2434.204944,2559.65,0,0,42.434977,2
46,10.07.2017 21:00:00.000,2385,2409,2256,2330,2.227143e+06,2388.396618,2523.3,2658.203382,10.692615,-0.216439,2658.203382,2388.396618,2541.05,0,0,40.352530,2
49,13.07.2017 21:00:00.000,2338,2356,2153,2160,2.321971e+06,2235.127432,2446.5,2657.872568,17.279589,-0.177713,2657.872568,2235.127432,2515.35,0,0,35.226683,2
53,19.07.2017 21:00:00.000,2255,2731,2245,2730,2.372186e+06,2125.859058,2362.8,2599.740942,20.055946,1.274877,2599.740942,2125.859058,2457.65,1,0,59.102687,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,02.10.2021 21:00:00.000,48077,49263,47185,48516,2.105904e+03,40372.602613,44353.4,48334.197387,17.950360,1.022835,48334.197387,40372.602613,45060.65,0,0,59.003128,1
1293,04.10.2021 21:00:00.000,49087,51831,48641,51534,2.166265e+03,40691.614864,45837.9,50984.185136,22.454280,1.053419,50984.185136,40691.614864,45346.30,0,0,65.437422,1
1294,05.10.2021 21:00:00.000,51564,55461,50481,55000,2.615271e+03,40531.162232,46969.3,53407.437768,27.414238,1.123682,53407.437768,40531.162232,45736.20,0,0,71.117209,1
1303,14.10.2021 21:00:00.000,57533,62922,56940,62587,2.235774e+03,52754.558444,56351.8,59949.041556,12.767087,1.366664,59949.041556,52754.558444,51094.85,2,0,76.431593,1


In [15]:
data['entry'] = 0

# Condition for entry category 2 (buy entry)
buy_entry_condition = (data['Trend'] == 2) & (data['RSI Signal'] == 2) & (data['Low'] < data['Lower Band'])
data.loc[buy_entry_condition, 'entry'] = 2

# Condition for entry category 1 (sell entry)
sell_entry_condition = (data['Trend'] == 1) & (data['RSI Signal'] == 1) & (data['High'] > data['Upper Band'])
data.loc[sell_entry_condition, 'entry'] = 1

In [16]:
data[data['entry']!=0]

,Gmt time,Open,High,Low,Close,Volume,BBL_10_1.5,BBM_10_1.5,BBU_10_1.5,BBB_10_1.5,BBP_10_1.5,Upper Band,Lower Band,SMA,Trend,entry,RSI,RSI Signal
53,19.07.2017 21:00:00.000,2255,2731,2245,2730,2.372186e+06,2125.859058,2362.8,2599.740942,20.055946,1.274877,2599.740942,2125.859058,2457.65,1,1,59.102687,1
89,07.09.2017 21:00:00.000,4598,4694,4140,4248,2.702564e+06,4286.459155,4521.3,4756.140845,10.388200,-0.081883,4756.140845,4286.459155,4365.20,2,2,54.968662,2
102,26.09.2017 21:00:00.000,3908,4193,3875,4154,2.035868e+06,3504.752665,3826.0,4147.247335,16.792856,1.010510,4147.247335,3504.752665,4111.60,1,1,54.393636,1
164,21.12.2017 22:00:00.000,15390,15811,10778,14245,3.911371e+05,14940.010563,16673.3,18406.589437,20.791198,-0.200489,18406.589437,14940.010563,14274.45,2,2,54.663540,2
202,14.02.2018 22:00:00.000,9337,10272,9302,9923,9.737800e+05,7435.937470,8534.8,9633.662530,25.750165,1.131653,9633.662530,7435.937470,9622.75,1,1,47.726577,1
242,11.04.2018 21:00:00.000,6964,8090,6833,7803,1.452145e+06,6528.054035,7023.7,7519.345965,14.113529,1.286146,7519.345965,6528.054035,7694.70,1,1,47.531463,1
263,10.05.2018 21:00:00.000,9158,9196,8449,8502,1.503795e+06,8782.904667,9291.8,9800.695333,10.953644,-0.275995,9800.695333,8782.904667,9001.95,2,2,44.965704,2
324,05.08.2018 21:00:00.000,7114,7217,6947,6982,2.784820e+03,7207.191204,7855.9,8504.608796,16.515200,-0.173569,8504.608796,7207.191204,7402.35,2,2,42.757304,2
338,23.08.2018 21:00:00.000,6480,6702,6479,6671,2.334246e+03,6220.160893,6431.3,6642.439107,6.565985,1.067635,6642.439107,6220.160893,6799.30,1,1,47.450490,1
339,26.08.2018 21:00:00.000,6739,6823,6715,6784,1.979413e+03,6219.399474,6476.2,6733.000526,7.930593,1.099298,6733.000526,6219.399474,6727.80,1,1,49.895240,1


## 3 - Entry based on a rejection candle next to Bollinger Bands

In [17]:
def identify_shooting_star(data):
    # Create a new column for shooting star
    data['shooting_star'] = data.apply(lambda row: 2 if (
        ( (min(row['Open'], row['Close']) - row['Low']) > (1.5 * abs(row['Close'] - row['Open']))) and 
        (row['High'] - max(row['Close'], row['Open'])) < (0.8 * abs(row['Close'] - row['Open'])) and 
        (abs(row['Open'] - row['Close']) > row['Open'] * 0.01)
    ) else 1 if (
        (row['High'] - max(row['Open'], row['Close'])) > (1.5 * abs(row['Open'] - row['Close'])) and 
        (min(row['Close'], row['Open']) - row['Low']) < (0.8 * abs(row['Open'] - row['Close'])) and 
        (abs(row['Open'] - row['Close']) > row['Open'] * 0.01)
    ) else 0, axis=1)

    return data

In [18]:
data = identify_shooting_star(data)

In [19]:
data[data['shooting_star']!=0]

,Gmt time,Open,High,Low,Close,Volume,BBL_10_1.5,BBM_10_1.5,BBU_10_1.5,BBB_10_1.5,BBP_10_1.5,Upper Band,Lower Band,SMA,Trend,entry,RSI,RSI Signal,shooting_star
26,12.06.2017 21:00:00.000,2657,2788,2638,2702,2.226144e+06,2342.284408,2620.0,2897.715592,21.199664,0.647633,2897.715592,2342.284408,2395.35,2,0,NaN,0,1
28,14.06.2017 21:00:00.000,2553,2559,2077,2363,2.589326e+06,2402.740468,2641.8,2880.859532,18.098231,-0.083118,2880.859532,2402.740468,2456.10,2,0,NaN,0,2
38,28.06.2017 21:00:00.000,2595,2605,2503,2559,2.302025e+06,2408.787342,2582.6,2756.412658,13.460285,0.432111,2756.412658,2408.787342,2612.20,0,0,50.080993,0,2
117,17.10.2017 21:00:00.000,5578,5613,5110,5508,2.096952e+06,4361.522285,5093.4,5825.277715,28.738278,0.783244,5825.277715,4361.522285,4571.00,2,0,70.757540,0,2
122,24.10.2017 21:00:00.000,5597,5694,5374,5681,3.314883e+06,5389.737882,5656.7,5923.662118,9.438794,0.545512,5923.662118,5389.737882,5050.80,2,0,65.874427,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1174,07.06.2021 21:00:00.000,34492,34667,31066,33673,2.251111e+03,34199.045276,36294.7,38390.354724,11.547993,-0.125509,38390.354724,34199.045276,36692.35,1,0,35.244876,2,2
1232,04.08.2021 21:00:00.000,39803,41414,37378,40960,1.547770e+03,38147.868767,39930.5,41713.131233,8.928670,0.788759,41713.131233,38147.868767,36326.95,2,0,64.659168,0,2
1243,15.08.2021 21:00:00.000,46767,48088,45720,46141,1.292448e+03,43963.869144,45730.0,47496.130856,7.724167,0.616356,47496.130856,43963.869144,43063.20,2,0,65.567518,0,1
1266,07.09.2021 21:00:00.000,46931,47405,44494,46116,2.135367e+03,46215.185242,49086.9,51958.614758,11.700534,-0.017269,51958.614758,46215.185242,48884.80,0,0,45.585602,2,2
